# タグ情報を利用した類似画像検索

In [ ]:
import os
import sys
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
import mysql.connector
import tensorflow as tf
import tensorflow_hub as hub
from annoy import AnnoyIndex
import matplotlib.pyplot as plt

In [ ]:
host = "localhost"
port = 3306
user = "root"
password = "humaninterface"
conn = mysql.connector.connect(
    host = host,
    port = port,
    user = user,
    password = password
)
image_dir = "/Volumes/Samsung_T5/19MH228/similar-image-retrieval/images/images_season_id_22"
image_ext = '.jpg'
image_path_list = glob.glob(image_dir + '/*' + image_ext)
image_ids = np.array([int(os.path.splitext(os.path.basename(image_path))[0]) for image_path in image_path_list])

In [ ]:
cur = conn.cursor(buffered=True)

distinct_color_statement = 'SELECT DISTINCT color FROM fp_production.image_details;'
cur.execute(distinct_color_statement)
distinct_colors = [row[0] for row in cur.fetchall()]
distinct_colors = distinct_colors if '' in distinct_colors else distinct_colors + ['']
color_dict = {}
for i, color in enumerate(distinct_colors):
    color_dict[color] = i 

distinct_materials_statement = 'SELECT DISTINCT material FROM fp_production.image_details;'
cur.execute(distinct_materials_statement)
distinct_materials = [row[0] for row in cur.fetchall()]
distinct_materials = distinct_materials if '' in distinct_materials else distinct_materials + ['']
material_dict = {}
for i, material in enumerate(distinct_materials):
    material_dict[material] = i 

distinct_patterns_statement = 'SELECT DISTINCT pattern FROM fp_production.image_details;'
cur.execute(distinct_patterns_statement)
distinct_patterns = [row[0] for row in cur.fetchall()]
distinct_patterns = distinct_patterns if '' in distinct_patterns else distinct_patterns + ['']
pattern_dict = {}
for i, pattern in enumerate(distinct_patterns):
    pattern_dict[pattern] = i 

distinct_item_categories_statement = 'SELECT DISTINCT item_category FROM fp_production.image_details;'
cur.execute(distinct_item_categories_statement)
distinct_item_categories = [row[0] for row in cur.fetchall()]
distinct_item_categories = distinct_item_categories if '' in distinct_item_categories else distinct_item_categories + ['']
item_category_dict = {}
for i, item_category in enumerate(distinct_item_categories):
    item_category_dict[item_category] = i 

distinct_item_groups_statement = 'SELECT DISTINCT item_group FROM fp_production.image_details;'
cur.execute(distinct_item_groups_statement)
distinct_item_groups = [row[0] for row in cur.fetchall()]
distinct_item_groups = distinct_item_groups if '' in distinct_item_groups else distinct_item_groups + ['']
item_group_dict = {}
for i, item_group in enumerate(distinct_item_groups):
    item_group_dict[item_group] = i 

distinct_items_statement = 'SELECT DISTINCT item FROM fp_production.image_details;'
cur.execute(distinct_items_statement)
distinct_items = [row[0] for row in cur.fetchall()]
distinct_items = distinct_items if '' in distinct_items else distinct_items + ['']
item_dict = {}
for i, item in enumerate(distinct_items):
    item_dict[item] = i 
    
item_information_dict = {
    'color': color_dict,
    'material': material_dict,
    'pattern': pattern_dict,
    'item_category': item_category_dict,
    'item_group': item_group_dict,
    'item': item_dict
}
print('item_information_dict -> {}\n'.format(item_information_dict))

In [ ]:
count = 0
for image_id in image_ids:
    print('image_id -> {}'.format(image_id))
    statement = "SELECT color, material, pattern, item_category, item_group, item, image_id FROM fp_production.image_details WHERE image_id = '{}';".format(image_id)
    cur = conn.cursor(buffered=True)
    cur.execute(statement)
    rows = cur.fetchall()
    for row in rows:
        color, material, pattern, item_category, item_group, item, image_id = row
        color_id = item_information_dict['color'][color]
        material_id = item_information_dict['material'][material]
        pattern_id = item_information_dict['pattern'][pattern]
        item_category_id = item_information_dict['item_category'][item_category]
        item_group_id = item_information_dict['item_group'][item_group]
        item_id = item_information_dict['item'][item]
        item_vector = [color_id, material_id, pattern_id, item_category_id, item_group_id, item_id]
        print('\titem_vector -> {}'.format(item_vector))
        count += 1

In [ ]:
collection_news_id_statement = 'SELECT id FROM fp_production.news WHERE category_sub="collection";'
cur.execute(collection_news_id_statement)
collection_news_ids = [row[0] for row in cur.fetchall()]
collection_image_id_statement = 'SELECT id FROM fp_production.images WHERE new_id IN ({});'.format(','.join([str(news_id) for news_id in collection_news_ids]))
cur.execute(collection_image_id_statement)
collection_image_ids = [row[0] for row in cur.fetchall()]
len(collection_image_ids)

In [ ]:
distinct_image_id_statement = 'SELECT DISTINCT image_id FROM fp_production.image_details;'
cur.execute(distinct_image_id_statement)
distinct_image_ids = [row[0] for row in cur.fetchall()]
len(distinct_image_ids)